In [12]:
pip install pydub

In [13]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import librosa
import soundfile
import os, glob, pickle
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pydub import AudioSegment
import wave
import contextlib

In [14]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(X, mfcc, chroma, mel,sample_rate):   
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        stft=np.abs(librosa.stft(X))
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [3]:
def predict_voice(file):
    x=[]
    y=[]
    y, s = librosa.load(file) # Downsample 44.1kHz to 8kHz
    feature=extract_feature(X=y, mfcc=True, chroma=True, mel=True,sample_rate=s)
    x.append(feature)
    y_pred=model.predict(x)
    emotion = y_pred[0]
    return emotion

In [5]:
#load the model 
model = pickle.load(open('voice_model.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neural_network. Anything that cannot be imported from sklearn.neural_network is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.21.2 when

## **Split Audio**

In [29]:
def get_duration(fname):
    with contextlib.closing(wave.open(fname,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        return duration

In [45]:
def get_emotion_dic(fname):
  duration = get_duration(fname) 
  # frames/ is the directory to stores audio frames
  dir='frames'
  if not os.path.exists(dir):
    os.makedirs(dir)
  else:
    shutil.rmtree(dir)           
    os.makedirs(dir)  
  emotion_dic={'bad':0,'medium':0,'good':0}
  originalAudio = AudioSegment.from_wav(fname)
  for i in range(0,int(duration),3):
      #newAudio = AudioSegment.from_wav("ted2.wav")
      newAudio = originalAudio[i*1000:(i+3)*1000]
      newAudio.export('frames/'+str(i)+'.wav', format="wav") #Exports to a wav file in the current path.
      emotion = predict_voice('frames/'+str(i)+'.wav') 
      emotion_dic[emotion]+=1
  return emotion_dic

In [49]:
get_emotion_dic('neutral.wav')

{'bad': 0, 'good': 0, 'medium': 1}

# Deploy

In [ ]:
app = Flask(__name__)
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    audio = request.files["audio"]
    print(audio)
    audiofile=audio.filename
    #dir of audio is in the current directory 
    emotion = predict_voice(audiofile)
    return emotion

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
